# TensorFlow MNIST チュートリアル
* TensorFlowでMNISTサンプルの学習・推論を行う備忘録。
* 必要なライブラリを読み込む
* 参考元
　　　　 * [MNIST For ML Beginners](https://www.tensorflow.org/versions/r0.10/tutorials/mnist/beginners/index.html)
    * [TensorFlow Mechanics 101](https://www.tensorflow.org/versions/master/tutorials/mnist/tf/index.html#tensorflow-mechanics-101)

## MLPでMNIST
* [MNIST For ML Beginners](https://www.tensorflow.org/versions/r0.10/tutorials/mnist/beginners/index.html)の内容

まずはデータを読み込む。
データは、以下のように分割されている。
* 55000件は、学習用データ(mnist.train)
* 10000件は、試験用データ(mnist.test)
* 5000件は、評価用データ(mnist.validation)
また、画像データをxとし、そのラベルをyとする。

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


* モデルを実装する。
    * placeholderはTensorFlowで計算を実行するときに利用する変数である。

In [2]:
import tensorflow as tf

In [3]:
x = tf.placeholder(tf.float32, [None, 784])

重み変数とバイアスについても定義する。

In [4]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

するとsoftmax回帰モデルは下記のように定義できる。

In [5]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

* 学習を行う。
    * モデルの良し悪しを計測するのは、交差エントロピーにて測る。
* 交差エントロピーを実装するために、新しいplaceholderを定義する。

In [6]:
y_ = tf.placeholder(tf.float32, [None, 10])

次いで交差エントロピーを定義する。

In [7]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y),  reduction_indices=[1]))

作成した変数の初期化をする演算を追加する。

In [8]:
init = tf.initialize_all_variables()

学習係数を0.5として、逆伝搬による学習のアルゴリズムを選択する。

In [11]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

セッションを作り、変数を初期化したのち、1000回学習を実施する。

In [12]:
sess = tf.Session()
sess.run(init)
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

* モデルを評価する。
    * 予測したラベルと正解ラベルが正しいか
    * 下記はbooleanのリストが返却されるので、平均を取ってみる。

In [15]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [16]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.919


## CNNでMNIST
* [Deep MNIST for Experts](https://www.tensorflow.org/versions/r0.9/tutorials/mnist/pros/index.html)の内容

* 重みとバイアスの初期化

In [19]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)
 
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

畳込み層とプーリング層を定義する

In [20]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
 
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

第１畳込み層を定義する。

In [21]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])

In [22]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

第２畳込み層を定義する。

In [23]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
 
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

全結合層を定義する。

In [25]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
 
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [26]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
 
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

* 最適化アルゴリズムとして、ADAM最適化を利用

In [27]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
 
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.12
step 100, training accuracy 0.84
step 200, training accuracy 0.92
step 300, training accuracy 0.96
step 400, training accuracy 0.9
step 500, training accuracy 0.92
step 600, training accuracy 1
step 700, training accuracy 0.98
step 800, training accuracy 1
step 900, training accuracy 0.98
step 1000, training accuracy 0.98
step 1100, training accuracy 0.94
step 1200, training accuracy 0.94
step 1300, training accuracy 0.96
step 1400, training accuracy 0.98
step 1500, training accuracy 0.96
step 1600, training accuracy 1
step 1700, training accuracy 0.98
step 1800, training accuracy 0.96
step 1900, training accuracy 0.96
step 2000, training accuracy 0.94
step 2100, training accuracy 1
step 2200, training accuracy 1
step 2300, training accuracy 1
step 2400, training accuracy 1
step 2500, training accuracy 0.96
step 2600, training accuracy 0.98
step 2700, training accuracy 1


KeyboardInterrupt: 

In [29]:
!python convolutional.py

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Initialized!
Step 0 (epoch 0.00), 9.6 ms
Minibatch loss: 12.053, learning rate: 0.010000
Minibatch error: 90.6%
Validation error: 84.6%
Step 100 (epoch 0.12), 955.2 ms
Minibatch loss: 3.283, learning rate: 0.010000
Minibatch error: 6.2%
Validation error: 6.8%
Step 200 (epoch 0.23), 1045.3 ms
Minibatch loss: 3.455, learning rate: 0.010000
Minibatch error: 17.2%
Validation error: 4.0%
Step 300 (epoch 0.35), 969.7 ms
Minibatch loss: 3.248, learning rate: 0.010000
Minibatch error: 7.8%
Validation error: 3.3%
Step 400 (epoch 0.47), 919.9 ms
Minibatch loss: 3.213, learning rate: 0.010000
Minibatch error: 7.8%
Validation error: 2.6%
Step 500 (epoch 0.58), 972.8 ms
Minibatch loss: 3.301, learning rate: 0.010000
Minibatch error: 9.4%
Validation error: 2.5%
Step 600 (epoch 0.70), 1072.3 ms
Minibatch loss: 3.118, learning rate: 0